# Results comparison between Trajectron++, the constant velocity model, and our method
Comparing the results of our method with the Trajectron++ paper's method and the constant velocity model.  

Our method and the CVM are evaluated live in this notebook.

Trajectron++ was trained and evaluated separately, the results reside in './trajectron++/results'. It was trained per the instructions in their repository: https://github.com/StanfordASL/Trajectron-plus-plus/tree/eccv2020. If needed then it can be trained and evaluated following their instructions again, the results should be copied to the same folder to make use of the new results. The data for evaluating our method is taken from the repository as well, this resides in './trajectron++/data'.  


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
np.random.seed(1)

## Helper functions

In [2]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    assert len(pred_x) == len(test_x)
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each pedestrian present in the current frame.
## Each node/pedestrian has to have at least 8 historical points and 12 future points.
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [3]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (8 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic
Logic for evaluating our method and the CVM

In [4]:
def evaluate_our_method(data, params, dataset_title='', clustering_method='KMeans', smoothing=False):
    our_fde_best_of = []
    our_ade_best_of = []
    our_fde_single = []
    our_ade_single = []

    # Loop over the dataset frame by frame
    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        
        # Loop over all agents in the current frame
        for node_id in pd.unique(frame_data.node_id):
            # Check for 8 points of history for current agent
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Check for 12 points of future for current agent
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, weights = generative_model.predict(x_data, y_data, params, trajectory_length=12, clustering_method=clustering_method, smoothing=smoothing)
                    assert len(all_pred_x[0]) == 12
                    
                    # This section is for producing a single trajectory
                    # Choose the first prediction (i.e. the most likely one)
                    single_x = all_pred_x[0]
                    single_y = all_pred_y[0]

                    single_fde = calculate_FDE(single_x, single_y, x_gt, y_gt)
                    single_ade = calculate_ADE(single_x, single_y, x_gt, y_gt)
                    our_fde_single.append(single_fde)
                    our_ade_single.append(single_ade)
                        
                        
                    # This section is for finding the best trajectories out of many in terms of FDE and ADE
                    best_fde = None
                    best_ade = None

                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of, our_fde_single, our_ade_single


In [5]:
# Evaluates CVM with scenarios 
# i.e. the CVM is run multiple times for one historical trajectory to attain many future predictions
# (variance is introduced by adding noise to the historical trajectory for each run)
def evaluate_cvm_with_scenarios(data, dataset_title='', history_length=8):
    # parameter history_length tells the CVM how many points of history it will use for 
    # calculating the constant velocity used for prediction
    
    cvm_fde_best_of = []
    cvm_ade_best_of = []  
    cvm_fde_single = []
    cvm_ade_single = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):
        frame_data = data[data.frame_id == frame_id]
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 8 historical points are present
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Check if at least 12 future points are present
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y = [], []
                    
                    # CVM
                    for i in range(20):
                        history_x = x_data[-history_length:]
                        history_y = y_data[-history_length:]
                        assert len(history_x) == history_length
                        
                        if i == 0:
                            vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                            vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]
                        else:
                            vel_x = [(history_x[i] - history_x[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_x))]
                            vel_y = [(history_y[i] - history_y[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_y))]
                        
                        assert len(vel_x) == history_length-1
                        avg_vel_x = np.mean(vel_x)
                        avg_vel_y = np.mean(vel_y)
                        
                        pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                        pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                        assert len(pred_x) == 12
                        
                        all_pred_x.append(pred_x)
                        all_pred_y.append(pred_y)
                        

                    best_fde = None
                    best_ade = None
                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade
                        
                        # The first prediction (without noise) counts as the result for the single prediction
                        if i == 0:
                            cvm_fde_single.append(fde)
                            cvm_ade_single.append(ade)

                    cvm_fde_best_of.append(best_fde)
                    cvm_ade_best_of.append(best_ade)
                    
    return cvm_fde_best_of, cvm_ade_best_of, cvm_fde_single, cvm_ade_single


## Automated results comparison

In [6]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [7]:
def evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, evaluate_most_likely=False):
    base_path = './trajectron++/data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_long_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_short_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for dataset_idx, dataset in enumerate(datasets):
        our_fde_bo20, our_ade_bo20 = [], []
        our_fde_most_likely, our_ade_most_likely = [], []
        
        cvm_fde_bo20, cvm_ade_bo20 = [], []
        cvm_short_fde_bo20, cvm_short_ade_bo20 = [], []
        
        cvm_fde_ml, cvm_ade_ml = [], []
        cvm_short_fde_ml, cvm_short_ade_ml = [], []
        
        for scene_idx, scene in enumerate(dataset):
            data = pd.read_csv(base_path + scene, sep='\t', index_col=False, header=None)
            data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

            data = process_data(data)

            ## Ours
            our_fde_best_of_20, our_ade_best_of_20, our_fde_single, our_ade_single = evaluate_our_method(data, our_method_params[dataset_idx], dataset_title=trajectron_resultset_names[dataset_idx], smoothing=False)
            our_fde_bo20 += our_fde_best_of_20
            our_ade_bo20 += our_ade_best_of_20
            if evaluate_most_likely:
                our_fde_most_likely += our_fde_single
                our_ade_most_likely += our_ade_single

            ## CVM
            cvm_fde_best_of, cvm_ade_best_of, cvm_fde_single, cvm_ade_single = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[dataset_idx])
            cvm_fde_bo20 += cvm_fde_best_of
            cvm_ade_bo20 += cvm_ade_best_of
            
            cvm_fde_short_history_best_of, cvm_ade_short_history_best_of, cvm_fde_short_history_single, cvm_ade_short_history_single = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[dataset_idx], history_length=2)
            cvm_short_fde_bo20 += cvm_fde_short_history_best_of
            cvm_short_ade_bo20 += cvm_ade_short_history_best_of

            if evaluate_most_likely:
                cvm_fde_ml += cvm_fde_single
                cvm_ade_ml += cvm_ade_single
                
                cvm_short_fde_ml += cvm_fde_short_history_single
                cvm_short_ade_ml += cvm_ade_short_history_single

        # add our and CVM results to the data dict
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_bo20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_bo20))
        if evaluate_most_likely:
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_most_likely))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_most_likely))
            
        cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_bo20))
        cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_bo20))
        cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_short_fde_bo20))
        cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_short_ade_bo20))
        if evaluate_most_likely:
            cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_ml))
            cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_ml))
            cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_short_fde_ml))
            cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_short_ade_ml))
                
        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx])
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx], suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        if len(dataset) == 1:
            num_predictable_trajectories = get_total_predictable_slices(data)
            assert len(trajectron_fde) == num_predictable_trajectories
            assert len(trajectron_ade) == num_predictable_trajectories
    
    return [
        ours_results,
        trajectron_results,
        cvm_long_results,
        cvm_short_results
    ]

### Parameters of our method for each dataset

In [8]:
eth_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 500, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.025, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.15, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}

hotel_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 500, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 11, 8]
}

zara1_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 500, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

zara2_params = {
    'NOISE': 0.1, 
    'NO_OF_TRAJECTORIES': 500, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.025, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.1, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

univ_params = {
    'NOISE': 0.025, 
    'NO_OF_TRAJECTORIES': 500, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}

### Running the evaluation

In [ ]:
our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]

datasets = [
    ['biwi_eth.txt'], 
    ['biwi_hotel.txt'], 
    ['students001.txt', 'students003.txt'],
    ['crowds_zara01.txt'], 
    ['crowds_zara02.txt'],
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel',
    'univ_vel',
    'zara1_vel', 
    'zara2_vel',
]

res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, evaluate_most_likely=True)


Ours - univ_vel:  28%|█████████████████████████▍                                                                 | 124/444 [05:53<15:42,  2.95s/it]

In [10]:
ours_results = res[0]
trajectron_results = res[1]
cvm_long_results = res[2]
cvm_short_results = res[3]

In [11]:
index = [
    'ETH', 
    'Hotel', 
    'Univ',
    'Zara 1', 
    'Zara 2',
]

df_data_best_of_20_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)
df_best_of_20_fde.loc['Average'] = df_best_of_20_fde.mean()

df_data_best_of_20_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)
df_best_of_20_ade.loc['Average'] = df_best_of_20_ade.mean()

df_data_most_likely_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
}

df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)
df_most_likely_fde.loc['Average'] = df_most_likely_fde.mean()

df_data_most_likely_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
}

df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)
df_most_likely_ade.loc['Average'] = df_most_likely_ade.mean()

# Results

### Best of 20 - FDE

In [12]:
df_best_of_20_fde.style.highlight_min(color = 'lightgreen', axis = 1).format(precision=3)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,1.134,1.726,0.812,0.660
Hotel,0.374,0.580,0.197,0.183
Univ,0.885,1.057,0.450,0.448
Zara 1,0.792,0.883,0.342,0.413
Zara 2,0.552,0.651,0.253,0.308
Average,0.747,0.979,0.411,0.402


### Best of 20 - ADE

In [13]:
df_best_of_20_ade.style.highlight_min(color = 'lightgreen', axis = 1).format(precision=3)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,0.647,0.892,0.396,0.462
Hotel,0.209,0.306,0.115,0.126
Univ,0.474,0.491,0.205,0.270
Zara 1,0.416,0.408,0.155,0.255
Zara 2,0.295,0.302,0.115,0.186
Average,0.408,0.480,0.197,0.260


### Single output - FDE

In [14]:
df_most_likely_fde.style.highlight_min(color = 'lightgreen', axis = 1).format(precision=3)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,2.303,2.282,1.615,2.036
Hotel,0.462,0.614,0.499,0.402
Univ,1.370,1.165,1.205,1.198
Zara 1,1.132,0.952,0.770,1.014
Zara 2,0.860,0.724,0.589,0.783
Average,1.226,1.148,0.936,1.087


### Single output - ADE

In [15]:
df_most_likely_ade.style.highlight_min(color = 'lightgreen', axis = 1).format(precision=3)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,1.102,1.075,0.695,0.961
Hotel,0.243,0.319,0.224,0.213
Univ,0.676,0.524,0.468,0.570
Zara 1,0.552,0.427,0.297,0.480
Zara 2,0.421,0.324,0.227,0.381
Average,0.599,0.534,0.382,0.521
